# Assembly functional enrichment

This notebook uses the output of the megahit assembly (see `MAG_assembly/mags.ipynb`), to produce clusters of functionally enriched proteins, following the 'blue' functional analysis process in Supplementary Figure S1. 

In brief, protein predictions are made from the assembled metagenomic contigs using prodigal, which are then clustered with MMSeqs2 and and annotated InterProScan. Abundance of the different proteins is then determined by mapping sequence reads back against the cDNA sequences using BWA. Eukaryotic sequences are screened out by alignment against Uniprot using diamond, following which enrichment analysis is carried out using DESeq2.

## Gene Prediction

First off, split the MegaHit final.contigs.fa into chunks so we can parallelise the prodigal predictions.

In [ ]:
%%bash 
bin/chunk_fasta.py --fasta megahit/final.contigs.fa --out megahit_chunked --chunks 50

Next, run prodigal predictions on these separate chunks, on the HPC cluster

So let's see how many predictions that gives us...

In [1]:
%%bash
grep -c '>' prodigal_predictions/final.contigs.*.proteins.fa|awk -F: '{sum+=$2} END {print sum}'

26740734


Now, created a merged file of all the predictions...

In [3]:
%%bash
for file in $(ls prodigal_predictions/*proteins.fa); do
    cat $file >> prodigal_predictions/prodigal.all.proteins.fa
done

In [9]:
%%bash
for file in $(ls prodigal_predictions/*nucl.fa); do
    cat $file >> prodigal_predictions/prodigal.all.nucl.fa
done

## Clustering

Cluster the protein predictions using MMSeqs2 to remove redundancy...

In [ ]:
%%bash
qsub bin/mmseqs_cluster.sh -i prodigal_predictions/prodigal.all.proteins.fa -o mmseqs_clusters

In [6]:
%%bash
grep -c '>' mmseqs_clusters/mmseqs_rep_seq.fasta

10554104


In [ ]:
And chunk the mmseqs representative sequences to parallelise subsequent interproscan runs

In [ ]:
%%bash
bin/chunk_fasta.py --fasta mmseqs_clusters/mmseqs_rep_seq.fasta --outdir mmseqs_chunked --chunks 1000

In [ ]:
The mmseqs clusters have sequences terminated with a * which is unacceptable to iprscan, so these need to go...

In [8]:
%%bash
sed -i 's/\*$//' mmseqs_chunked/*

## Read mapping to cDNA sequences

The cDNA sequences represeting the cluster protein sequences have the sequence reads mapped against them for later assessment. Firstly we need to obtain the necessary cDNA sequences...

In [ ]:
%%bash
qsub bin/get_rep_cdna.py --qsub bin/get_rep_cdna.py --repprot mmseqs_clusters/mmseqs_rep_seq.fasta --repcdna mmseqs_clusters/mmseqs_rep_cDNA_seq.fasta --cdna prodigal_predictions/prodigal.all.nucl.fa

In [ ]:
Now the reads can be mapped against these...

## INTERPRO annotation

InterProScan is used to obtain GO annotations for the clustered protein predictions. 

We use Snakemake to manage this process, using DRMAA to submit jobs to a UGE-based cluster, which may require customisation for other environments. These changes may be required in both the `run_workflow.sh` script, `config.json` and also the GridEngine directives beginning with `#$` in any scrips in the `bin` directory.

The `bin/interpro.sh` script will place a copy of the interpro database onto the disk of the execution hosts, if one is not already present, so local disks with sufficient capacity are required on cluster nodes for this approach to work. The names of nodes which have a copy of the database are stored in a local text file. Once all jobs have successfully completed, the `bin/submit_cleanup.sh` script will submit jobs to each of these nodes to remove the database copies.

The Snakemake workflow incorporates additional post-annotation parsing, downloading and reformatting GO, counting reads mapped to cDNA sequences and quantifying GO terms, and carrying out GO Slim mapping.

The whole process results in the output of abundances of GO and GO Slim 

In [ ]:
#%%bash
mkdir -p interproscan/inputs
cd interproscan/inputs

for chunk in $(ls ../../mmseqs_chunked/*fa); do
echo $chunk
    ln -s $chunk
done
cd ..